# Data Loading, Storage, and File Formats

- 참고 사이트: 데이터과학(Python 사용)에서 일반적으로 사용되는 파일 형식을 읽는 방법
- 참고문헌
    - Python for Data Analysis (Ch.6), Wes Mckinney, O'Reily, 2nd Edi., 2017.
    - pandas: powerful Python data analysis toolkit, Release 0.23.4, 2016
    - [https://www.nextobe.com]

In [7]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 1. Reading and Writing Data in Text Format
- table data를 DataFrame 객체로 읽기
![표6-1](img/table6-1-1.png)
![표6-1](img/table6-1-2.png)

- examples/ex1.csv 파일 내용
```
a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
```

In [8]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
# read_table, delimiter 지정
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- examples/ex2.csv 파일 내용 (no header row)
```
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
```

In [10]:
pd.read_csv('examples/ex2.csv', header=None) # header를 통해 column에 대한 index를 지정하는 것이 아니라 순서대로 indexing한다

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
# index_col 옵션: index column 지정
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


- examples/csv_mindex.csv 내용
![cvs_mindex](img/cvs_mindex.png)

In [13]:
# hierarchical index: index_col에 여러 column list
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [14]:
# table이 field를 구분하는 특정 구분자(delimeter)가 없는 경우
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [15]:
# delimiter로 정규표현식 사용
# 행의 데이터 갯수보다 한개 적은 column name을 가진 경우, 첫번째 열을 index로 추론
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [16]:
list(open('examples/ex4.csv'))

['# hey!\n',
 'a,b,c,d,message\n',
 '# just wanted to make things more difficult for you\n',
 '# who reads CSV files with computers, anyway?\n',
 '1,2,3,4,hello\n',
 '5,6,7,8,world\n',
 '9,10,11,12,foo']

In [17]:
# 특정 row skip(data에 포함시키지 않음)
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [18]:
list(open('examples/ex5.csv'))

['something,a,b,c,d,message\n',
 'one,1,2,3,4,NA\n',
 'two,5,6,,8,world\n',
 'three,9,10,11,12,foo']

In [19]:
# empty string 혹은 sentinel value(주로 NA, NULL) 처리
result = pd.read_csv('examples/ex5.csv')
result
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [20]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [21]:
# dictionary를 이용한 sentinel value 지정
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


![표6-2](img/table6-2-1.png)
![표6-2](img/table6-2-2.png)

## 1.1 Text Files의 일부만 읽기

In [22]:
# 화면에 출력될 행의 수 지정
pd.options.display.max_rows = 10

In [23]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [24]:
# nrows: 읽을 행의 수 지정
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [25]:
# chunksize: 읽을 행의 수 지정
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [26]:
# 1000줄 읽은 chunker에서 key 값의 빈도 계산
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

# 빈도 순으로 내림차순 정렬
    tot = tot.sort_values(ascending=False)

C:\Users\samsung\AppData\Local\Temp\ipykernel_4420\1695306430.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [27]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 1.2 Data를 Text Format으로 쓰기

In [28]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [29]:
# comma-separated file
data.to_csv('examples/out.csv')
list(open('examples/out.csv'))

[',something,a,b,c,d,message\n',
 '0,one,1,2,3.0,4,\n',
 '1,two,5,6,,8,world\n',
 '2,three,9,10,11.0,12,foo\n']

In [30]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [31]:
# NaN 값을 특정 값으로 처리
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [32]:
# row의 index와 column의 label 출력 안되게
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [33]:
# column의 일부만 출력(지정된 순서)
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [34]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [35]:
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [36]:
ts.to_csv('examples/tseries.csv')
list(open('examples/tseries.csv'))

[',0\n',
 '2000-01-01,0\n',
 '2000-01-02,1\n',
 '2000-01-03,2\n',
 '2000-01-04,3\n',
 '2000-01-05,4\n',
 '2000-01-06,5\n',
 '2000-01-07,6\n']

## 1.3 Working with Delimited Formats

In [37]:
list(open('examples/ex7.csv'))

['"a","b","c"\n', '"1","2","3"\n', '"1","2","3"\n']

- 표 형식의 data는 pandas.read_table로 읽음
- 형식이 잘못된 파일 읽기 => python의 csv module 사용

In [38]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [39]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [40]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [41]:
header, values = lines[0], lines[1:]
values

[['1', '2', '3'], ['1', '2', '3']]

In [42]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

- 새로운 format 정의(delimiter, string quoting convention, line terminator 등)
![p177](img/subclass_csv.Dialect.PNG)

![표6-3](img/table6-3-1.png)
![표6-3](img/table6-3-2.png)

- csv.writer: delimited file 저장
![p178](img/csv_writer_p178.PNG)

## 1.4 JSON Data
- JSON: JavaScript Object Notation, web browser와 application 사이에 HHTP request에 의한 data 전송 format
- *null*: null value
- key는 string만 됨
- json.load: conver a JSON string to Python form
- json.dump: convert a Python object to JSON
- 참조: jump2python e-Book 7.5

In [43]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [44]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [45]:
# python object => JSON
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [46]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


- pandas.read_json: JSON dataset을 Series나 DataFrame으로 자동 변경

- examples/example.json 내용
![example](img/example_jason.PNG)

In [47]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [48]:
# export data from pandas to JSON
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## 1.5 XML and HTML: Web Scraping
- lxml, beautifulsoup4, html5lib: XML, HTML의 data 읽고 쓰는 library
- pip install lxml
- pip install beautifulsoup4 html5lib

- pandas.read_html:  <table> tag 내에 포함된 모든 테이블 data 만 추출
- 참조: jump2python e-Book 7.4

In [49]:
# fdic_failed_bank_list.html 확인
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [50]:
len(tables)

1

In [51]:
failures = tables[0]

In [52]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [53]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

# 2. Binary Data Formats
- Python pickle serialization 내장함수
- pandas 데이터 저장: **to_pickle** method 사용
- 저장된 pickle 데이터 읽기: **read_pickle**

- memory data 그자체를 사용하기 위해서 binary data를 사용한다 (read, write를 위해선 변환이 필요하기 때문)

In [54]:
frame = pd.read_csv('examples/ex1.csv')
frame
frame.to_pickle('examples/frame_pickle')

In [55]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 2.1 Using HDF5 Format
- HDF: Hierarchical Data Format
- 대규모 과학용 배열 data 저장
- 다양한 datasets 저장, metadata 지원
- 다양한 압축기술을 사용하여 상황에맞게(on-the-fly) 압축 지원
- **PyTables, h5py** library를 사용하여 HDF5 파일 access

In [56]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [57]:
frame

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [58]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [59]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [60]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


## 2.2 Reading/Writing Microsoft Excel Files

In [61]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [62]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [63]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [64]:
# write pandas data to Excel format
# ExcelWriter 생성 후 to_excel method
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [65]:
# 직접 to_excel에 직접 파일이름 전달
frame.to_excel('examples/ex2.xlsx')

# 3. Interacting with Web APIs

- request package 사용하여 Python에서 Web API 접근

In [66]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

- https://api.github.com/repos/pandas-dev/pandas/issues 내용
![github_pandas_issues](img/github_pandas_issues.PNG)

In [67]:
data = resp.json()
data[0]['title']

'Update doc URL in `setup.cfg` (for `1.5.x`)'

In [68]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,50279,Update doc URL in `setup.cfg` (for `1.5.x`),[],open
1,50278,Update doc URL in `pyproject.toml`,[],open
2,50277,Backport PR #50117 on branch 1.5.x (Alternate ...,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
3,50276,QST: AttributeError: can't set attribute 'styl...,"[{'id': 34444536, 'node_id': 'MDU6TGFiZWwzNDQ0...",open
4,50275,(wait til 2.0) STYLE Reduce merge conflicts wi...,"[{'id': 106935113, 'node_id': 'MDU6TGFiZWwxMDY...",open
...,...,...,...,...
25,50247,ENH: Add use_nullable_dtypes to read_sql_query...,"[{'id': 47232590, 'node_id': 'MDU6TGFiZWw0NzIz...",open
26,50246,Disallow cumprod for timedelta,"[{'id': 49597148, 'node_id': 'MDU6TGFiZWw0OTU5...",open
27,50245,BUG: When read_sql_query() returns no records ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,50244,BUG: assign with apply over axis=1 sometimes f...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


# 4. Interacting with Databases

In [69]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

OperationalError: table test already exists

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description

In [ ]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)